In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import os
# from PIL import Image
# import numpy as np

# def load_data(data_dir):
#     images = []
#     labels = []
#     classes = os.listdir(data_dir)
#     for cls in classes:
#         cls_path = os.path.join(data_dir, cls)
#         if os.path.isdir(cls_path):
#             cls_images = os.listdir(cls_path)
#             for img_name in cls_images:
#                 img_path = os.path.join(cls_path, img_name)
#                 img = Image.open(img_path).resize((224, 224))
#                 img = np.array(img) / 255.0
#                 images.append(img)
#                 labels.append(cls)
#     return np.array(images), np.array(labels)


# data_dir = '/kaggle/input/inaturalist12k/Data/inaturalist_12K'

# train_dir = os.path.join(data_dir, 'train')
# X_train, y_train = load_data(train_dir)

# val_dir = os.path.join(data_dir, 'val')
# X_val, y_val = load_data(val_dir)


In [ ]:
# print(os.listdir('/kaggle/input'))

In [4]:
import os
from PIL import Image
import numpy as np
#import tensorflow as tf
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch
import wandb
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_dir = '/kaggle/input/inaturalist12k/Data/inaturalist_12K'
#-------------------------------------------------------------------------------------------------------
def get_transforms(data_augmentation):
    if data_augmentation=="yes":
        transform = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            transforms.RandomRotation(degrees=30),
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
            transforms.Resize((256,256)),
            transforms.ToTensor()
        ])
    else:
        transform = transforms.Compose([
            transforms.Resize((256,256)),
            transforms.ToTensor()
        ])
    return transform

transform = transforms.Compose([transforms.Resize((256,256)),transforms.ToTensor()])
train_dataset=torchvision.datasets.ImageFolder(root=data_dir+'/train',transform=get_transforms("yes"))
test_dataset=torchvision.datasets.ImageFolder(root=data_dir+'/val',transform=transform)

val_size = int(len(train_dataset) * 0.2)
train_size = len(train_dataset) - val_size

train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader=DataLoader(train_dataset,batch_size=16,shuffle=True)
val_loader=DataLoader(val_dataset,batch_size=16,shuffle=False)
test_loader=DataLoader(test_dataset,batch_size=16,shuffle=False)
#--------------------------------------------------------------------------------------------------------

In [5]:
import torch.nn as nn
import torch.optim as optim

import torch.nn.functional as F
class Mish(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        return x * torch.tanh(F.softplus(x))
# Define the CNN model architecture
class CNN(nn.Module):
    def __init__(self, num_filters, filter_size, activation_fn, filter_org, batch_norm, dropout,num_neuron_fc,filter_size_org):
        super(CNN, self).__init__()
        if(activation_fn=="ReLU"):
            activation_fn=nn.ReLU
        if(activation_fn=="GELU"):
            activation_fn=nn.GELU
        if(activation_fn=="SiLU"):
            activation_fn=nn.SiLU
        if(activation_fn=="Mish"):
            activation_fn=Mish
        if(filter_org=="double"):
            filter_org=[1,2,2,2,2]
        if(filter_org=="same"):
            filter_org=[1,1,1,1,1]
        if(filter_org=="half"):
            filter_org=[1,0.5,0.5,0.5,0.5]
        if(filter_size_org=='same'):
            filter_size_org=[1,1,1,1,1]
        if(filter_size_org=="double"):
            filter_size_org=[1,2,2,2,2]
        if(filter_size_org=="half"):
            filter_size_org=[1,0.5,0.5,0.5,0.5]
        layers = []
        in_channels = 3
        w=256
        for i, f in enumerate(filter_org):
            out_channels = int(num_filters * f)
            filter_size1 = int(filter_size*filter_size_org[i])
            #calculate feature map dimension
            w=int((w-filter_size1+(2*1))+1)
            w=int(((w-2)//2)+1)
            #ends
            layers.append(nn.Conv2d(int(in_channels), int(out_channels), int(filter_size1), padding=1))
            if batch_norm:
                layers.append(nn.BatchNorm2d(out_channels))
            layers.append(activation_fn())
            layers.append(nn.MaxPool2d(2))
            if dropout > 0:
                layers.append(nn.Dropout(dropout))
            in_channels = out_channels
        #print(w)
        self.cnn = nn.Sequential(*layers)      
        self.fc1 = nn.Linear(int(out_channels) * w * w, num_neuron_fc)
        self.fc2 = nn.Linear(num_neuron_fc, 10)

    def forward(self, x):
        x = self.cnn(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x



# #------------------------------------------------------------------------------------------------------
# sweep_config = {
#     'method': 'random', #grid, random,#bayes
#     'name' : 'Random_sweep_cross_entropy',
#     'metric': {
#       'name': 'valid accuracy',
#       'goal': 'maximize'  
#     },
#     'parameters': {
#         'num_filters': {
#             'values': [32,64,96]
#         },
#         'filter_size':{
#             'values':[3,5,7,2]
#         },
#          'activation_fn':{
#             'values':["Mish","ReLU","GELU","SiLU"]
#         },
#         'filter_org':{
#             'values':["same","double","half"]
#         },
#          'filter_size_org':{
#             'values':["same","double"]
#         },
#          'batch_norm': {
#             'values': [True,False]
#         },
#         'dropout': {
#             'values': [0.3,0.2]
#         },
#          'num_neuron_fc':{
#             'values':[10,20]
#         },
#         'data_augmentation': {
#             'values': ["no","yes"]
#         }, 
# #         'loss': {
# #             'values': ['entropy_loss']
# #         }, 
        
        
#     }
# }

# sweep_id = wandb.sweep(sweep=sweep_config, project='DL_Assign_2')
# from types import SimpleNamespace
# def main():
#     with wandb.init() as run:
#         params={}
#         params=dict(wandb.config)
#         params=SimpleNamespace(**params)
#         run_name="num_filter_"+str(wandb.config.num_filters)+"-filter_size_"+str(wandb.config.filter_size)+"-activation_fn_"+wandb.config.activation_fn\
#                 + "-filter_org"+wandb.config.filter_org+"-filter_size_org"+wandb.config.filter_size_org+"-batch_norm"+str(wandb.config.batch_norm)\
#                 + "-dropout"+str(wandb.config.dropout)+"num_neuron_fc"+str(wandb.config.num_neuron_fc)+"-data_augmentation"+wandb.config.data_augmentation
#         wandb.run.name=run_name
        
#         #data augmentation function starts
#         def get_transforms(data_augmentation):
#             if data_augmentation=="yes":
#                 transform = transforms.Compose([
#                     transforms.RandomHorizontalFlip(p=0.5),
#                     transforms.RandomVerticalFlip(p=0.5),
#                     transforms.RandomRotation(degrees=30),
#                     transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
#                     transforms.Resize((256,256)),
#                     transforms.ToTensor()
#                 ])
#             else:
#                 transform = transforms.Compose([
#                     transforms.Resize((256,256)),
#                     transforms.ToTensor()
#                 ])
#             return transform
#         #data augmentation function ends
        
#         #transform and splitting data starts
#         transform = transforms.Compose([transforms.Resize((256,256)),transforms.ToTensor()])
#         train_dataset=torchvision.datasets.ImageFolder(root=data_dir+'/train',transform=get_transforms(wandb.config.data_augmentation))
#         test_dataset=torchvision.datasets.ImageFolder(root=data_dir+'/val',transform=transform)

#         val_size = int(len(train_dataset) * 0.2)
#         train_size = len(train_dataset) - val_size

#         train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

#         train_loader=DataLoader(train_dataset,batch_size=64,shuffle=True)
#         val_loader=DataLoader(val_dataset,batch_size=64,shuffle=False)
#         test_loader=DataLoader(test_dataset,batch_size=64,shuffle=False)
#         #transform and splitting data ends
        
#         # Initialize the model
#         model = CNN(params.num_filters, params.filter_size, params.activation_fn, params.filter_org, params.batch_norm, params.dropout, params.num_neuron_fc,params.filter_size_org).to(device)

#         # Define the loss function and optimizer
#         criterion = nn.CrossEntropyLoss()
#         optimizer = optim.Adam(model.parameters(), lr=0.001)

#         # Train the model
#         num_epochs = 10
#         for epoch in range(num_epochs):
#             model.train()
#             for batch_idx, (data, target) in enumerate(train_loader):
#                 data, target = data.to(device), target.to(device)# Move data and target to the device
#                 optimizer.zero_grad()
#                 output = model(data)
#                 loss = criterion(output, target)
#                 loss.backward()
#                 optimizer.step()

#             # Evaluate the model on train_loader
#             model.eval()
#             train_correct = 0
#             train_total = 0
#             train_loss = 0.0
#             with torch.no_grad():
#                 for data, target in train_loader:
#                     data, target = data.to(device), target.to(device)# Move data and target to the device
#                     output = model(data)
#                     loss = criterion(output, target)
#                     train_loss += loss.item()
#                     _, predicted = torch.max(output.data, 1)
#                     train_total += target.size(0)
#                     train_correct += (predicted == target).sum().item()

#             train_accuracy = 100 * train_correct/train_total
#             train_loss /= len(train_loader)

#             # Set model to evaluation mode
#             model.eval()
#             val_loss = 0.0
#             correct = 0
#             total = 0
#             with torch.no_grad():
#                 for batch_idx, (data, target) in enumerate(val_loader):
#                     data, target = data.to(device), target.to(device)# Move data and target to the device
#                     output = model(data)
#                     loss = criterion(output, target)
#                     val_loss += loss.item()
#                     _, predicted = torch.max(output.data, 1)
#                     total += target.size(0)
#                     correct += (predicted == target).sum().item()

#             # Calculate validation metrics
#             val_loss /= len(val_loader)
#             val_accuracy = 100 * correct / total

#             # Print metrics for current epoch
#             #print('Epoch: {} \t Training Loss: {:.6f}
#             print("epoch",epoch)
#             print('Training Loss: {:.6f} \t Training Accuracy: {:.6f}'.format(train_loss, train_accuracy))
#             print('Validation Loss: {:.6f} \t Validation Accuracy: {:.6f}'.format(val_loss, val_accuracy))
#             wandb.log({'train loss':train_loss,'train accuracy':train_accuracy,'valid loss':val_loss,'valid accuracy':val_accuracy})
# #         input_size=784
# #         output_size=10
# #         obj=neural_network(input_size,output_size,params)
# #         obj.predict()

    
# wandb.agent(sweep_id, function=main,count=35)

#----------------------------------------------------------------------------------------------------


#Define the model hyperparameters
num_filters = 96
filter_size = 5
activation_fn = "ReLU"
filter_org = "same"
filter_size_org="double" # extra added
batch_norm = False
dropout = 0.3
num_neuron_fc=10


# Initialize the model
model = CNN(num_filters, filter_size, activation_fn, filter_org, batch_norm, dropout, num_neuron_fc,filter_size_org).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)# Move data and target to the device
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    
    # Evaluate the model on train_loader
    model.eval()
    train_correct = 0
    train_total = 0
    train_loss = 0.0
    with torch.no_grad():
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)# Move data and target to the device
            output = model(data)
            loss = criterion(output, target)
            train_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            train_total += target.size(0)
            train_correct += (predicted == target).sum().item()

    train_accuracy = 100 * train_correct/train_total
    train_loss /= len(train_loader)
    
    # Set model to evaluation mode
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(val_loader):
            data, target = data.to(device), target.to(device)# Move data and target to the device
            output = model(data)
            loss = criterion(output, target)
            val_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
            
    # Calculate validation metrics
    val_loss /= len(val_loader)
    val_accuracy = 100 * correct / total
    
    # Print metrics for current epoch
    #print('Epoch: {} \t Training Loss: {:.6f}
    print("epoch",epoch)
          
    print('Training Loss: {:.6f} \t Training Accuracy: {:.6f}'.format(train_loss, train_accuracy))
    print('Validation Loss: {:.6f} \t Validation Accuracy: {:.6f}'.format(val_loss, val_accuracy))




#------------------------------------------------------------------------------------------
    
    
    
    
    
#     train_loss = 0.0
#     for batch_idx, (data, target) in enumerate(train_loader): 
#         data, target = data.to(device), target.to(device)# Move data and target to the device
#         optimizer.zero_grad()
#         output = model(data)
#         loss = criterion(output, target)
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.item()

#     print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss/len(train_loader)))
    
#     # Evaluate the model on train_loader
#     model.eval()
#     train_correct = 0
#     train_total = 0
#     train_loss = 0.0
#     with torch.no_grad():
#         for data, target in train_loader:
#             output = model(data)
#             loss = criterion(output, target)
#             train_loss += loss.item()
#             _, predicted = torch.max(output.data, 1)
#             train_total += target.size(0)
#             train_correct += (predicted == target).sum().item()

#     train_accuracy = train_correct/train_total
#     train_loss /= len(train_loader)

#     # Evaluate the model on val_loader
#     model.eval()
#     val_correct = 0
#     val_total = 0
#     val_loss = 0.0
#     with torch.no_grad():
#         for data, target in val_loader:
#             output = model(data)
#             loss = criterion(output, target)
#             val_loss += loss.item()
#             _, predicted = torch.max(output.data, 1)
#             val_total += target.size(0)
#             val_correct += (predicted == target).sum().item()

#     val_accuracy = val_correct/val_total
#     val_loss /= len(val_loader)

#     print('Training Loss: {:.6f} \t Training Accuracy: {:.6f}'.format(train_loss, train_accuracy))
#     print('Validation Loss: {:.6f} \t Validation Accuracy: {:.6f}'.format(val_loss, val_accuracy))



RuntimeError: Calculated padded input size per channel: (23 x 23). Kernel size: (40 x 40). Kernel size can't be greater than actual input size

In [ ]:
# import torch
# import matplotlib.pyplot as plt

# # Load an image into a PyTorch tensor
# for i in range(1):
#     img=train_loader[i][0].numpy().transpose((1,2,0))
    
# # img_tensor = torch.randn(3, 256, 256)

# # # Convert the tensor to a NumPy array
# # img = img_tensor.numpy().transpose((1, 2, 0))

# plt.imshow(grid.permute(1, 2, 0))

# # Display the image
#     plt.imshow(img)
#     plt.show()

In [ ]:
# import os
# from PIL import Image
# import numpy as np
# #import tensorflow as tf
# import torchvision
# import torchvision.transforms as transforms
# from torch.utils.data import DataLoader
# from torch.utils.data import random_split
# import torch
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# data_dir = '/kaggle/input/inaturalist12k/Data/inaturalist_12K'
# transform = transforms.Compose([transforms.Resize((256,256)),transforms.ToTensor()])

# train_dataset=torchvision.datasets.ImageFolder(root=data_dir+'/train',transform=transform)
# test_dataset=torchvision.datasets.ImageFolder(root=data_dir+'/val',transform=transform)

# val_size = int(len(train_dataset) * 0.5)
# train_size = len(train_dataset) - val_size

# train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# train_loader=DataLoader(train_dataset,batch_size=64,shuffle=True)
# val_loader=DataLoader(val_dataset,batch_size=64,shuffle=False)
# test_loader=DataLoader(test_dataset,batch_size=64,shuffle=False)






# import torch.nn as nn
# import torch.optim as optim

# # Define the CNN model architecture
# class CNN(nn.Module):
#     def __init__(self, num_filters, filter_size, activation_fn, filter_org, batch_norm, dropout):
#         super(CNN, self).__init__()
#         layers = []
#         in_channels = 3
#         for i, f in enumerate(filter_org):
#             out_channels = num_filters * f
#             layers.append(nn.Conv2d(in_channels, out_channels, filter_size, padding=1))
#             if batch_norm:
#                 layers.append(nn.BatchNorm2d(out_channels))
#             layers.append(activation_fn())
#             layers.append(nn.MaxPool2d(2))
#             if dropout > 0:
#                 layers.append(nn.Dropout(dropout))
#             in_channels = out_channels
#         self.cnn = nn.Sequential(*layers)
#         self.fc = nn.Linear(out_channels * 32 * 32, 10)

#     def forward(self, x):
#         x = self.cnn(x)
#         x = x.view(x.size(0), -1)
#         x = self.fc(x)
#         return x

# # Define the model hyperparameters
# num_filters = 32
# filter_size = 3
# activation_fn = nn.ReLU
# filter_org = [1,2,2]
# batch_norm = True
# dropout = 0.3

# # Initialize the model
# model = CNN(num_filters, filter_size, activation_fn, filter_org, batch_norm, dropout).to(device)

# # Define the loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Train the model
# num_epochs = 5
# for epoch in range(num_epochs):
#     model.train()
#     train_loss = 0.0
#     correct = 0
#     total = 0
#     for batch_idx, (data, target) in enumerate(train_loader):
#         data, target = data.to(device), target.to(device)# Move data and target to the device
#         optimizer.zero_grad()
#         output = model(data)
#         loss = criterion(output, target)
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.item()
#         _, predicted = torch.max(output.data, 1)
#         total += target.size(0)
#         correct += (predicted == target).sum().item()
        
#     # Calculate training metrics
#     train_loss /= len(train_loader)
#     train_accuracy = 100 * correct / total
    
#     # Set model to evaluation mode
#     model.eval()
#     val_loss = 0.0
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for batch_idx, (data, target) in enumerate(val_loader):
#             data, target = data.to(device), target.to(device)# Move data and target to the device
#             output = model(data)
#             loss = criterion(output, target)
#             val_loss += loss.item()
#             _, predicted = torch.max(output.data, 1)
#             total += target.size(0)
#             correct += (predicted == target).sum().item()
            
#     # Calculate validation metrics
#     val_loss /= len(val_loader)
#     val_accuracy = 100 * correct / total
    
#     # Print metrics for current epoch
#     #print('Epoch: {} \t Training Loss: {:.6f}
#     print("epoch",epoch)
          
#     print('Training Loss: {:.6f} \t Training Accuracy: {:.6f}'.format(train_loss, train_accuracy))
#     print('Validation Loss: {:.6f} \t Validation Accuracy: {:.6f}'.format(val_loss, val_accuracy))





    
    
    
    
    
# #     train_loss = 0.0
# #     for batch_idx, (data, target) in enumerate(train_loader): 
# #         data, target = data.to(device), target.to(device)# Move data and target to the device
# #         optimizer.zero_grad()
# #         output = model(data)
# #         loss = criterion(output, target)
# #         loss.backward()
# #         optimizer.step()
# #         train_loss += loss.item()

# #     print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss/len(train_loader)))
    
# #     # Evaluate the model on train_loader
# #     model.eval()
# #     train_correct = 0
# #     train_total = 0
# #     train_loss = 0.0
# #     with torch.no_grad():
# #         for data, target in train_loader:
# #             output = model(data)
# #             loss = criterion(output, target)
# #             train_loss += loss.item()
# #             _, predicted = torch.max(output.data, 1)
# #             train_total += target.size(0)
# #             train_correct += (predicted == target).sum().item()

# #     train_accuracy = train_correct/train_total
# #     train_loss /= len(train_loader)

# #     # Evaluate the model on val_loader
# #     model.eval()
# #     val_correct = 0
# #     val_total = 0
# #     val_loss = 0.0
# #     with torch.no_grad():
# #         for data, target in val_loader:
# #             output = model(data)
# #             loss = criterion(output, target)
# #             val_loss += loss.item()
# #             _, predicted = torch.max(output.data, 1)
# #             val_total += target.size(0)
# #             val_correct += (predicted == target).sum().item()

# #     val_accuracy = val_correct/val_total
# #     val_loss /= len(val_loader)

# #     print('Training Loss: {:.6f} \t Training Accuracy: {:.6f}'.format(train_loss, train_accuracy))
# #     print('Validation Loss: {:.6f} \t Validation Accuracy: {:.6f}'.format(val_loss, val_accuracy))

